<a href="https://colab.research.google.com/github/wadeyman/introduction-to-tensorflow/blob/main/Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import numpy as np
import plotly

https://www.youtube.com/watch?v=BQ4kVn-Rt84

In [8]:
def target_function(individuals):
  result = []
  for x in individuals:
    result.append(np.exp(-(x[0] - 2)**2) + np.exp(-(x[0] -6)**2/10) + 1/(x[0]**2 + 1))
  return torch.tensor(result)


$e^{-(x-2)^2}+e^{-(x-6)^{2}/10} + 1/(x^2 + 1)$

In [12]:
np.exp(-(0 - 2)**2)

0.01831563888873418

In [9]:
import plotly.graph_objects as go

x = np.linspace(-2., 10., 100)

x_new = x.reshape((100,-1)) #100 row, 1 column
z = target_function(x_new)

x_new[0:5]

array([[-2.        ],
       [-1.87878788],
       [-1.75757576],
       [-1.63636364],
       [-1.51515152]])

In [10]:
data = go.Scatter(x=x , y=z, line_color ="red")
fig = go.Figure(data = data)
fig.update_layout(title="Example Test Function", xaxis_title = "x input",
                  yaxis_title='output')
fig.show()

In [14]:
train_x = torch.rand(10,1)
train_x

tensor([[0.1628],
        [0.7058],
        [0.9229],
        [0.5372],
        [0.6816],
        [0.6656],
        [0.7900],
        [0.1627],
        [0.8900],
        [0.3804]])

In [15]:
exact_obj = target_function(train_x).unsqueeze(-1)
# get the output based on random x values
exact_obj

tensor([[1.0415],
        [0.9154],
        [0.9294],
        [0.9443],
        [0.9177],
        [0.9196],
        [0.9133],
        [1.0415],
        [0.9231],
        [0.9887]])

In [17]:
best_observed_value = exact_obj.max().item()

In [30]:
def gen_init_data(n=10):
  train_x = torch.rand(n, 1)
  exact_obj = target_function(train_x).unsqueeze(-1)
  best_observed_value = exact_obj.max().item()
  return train_x, exact_obj, best_observed_value

In [31]:
init_x, init_y, best_init_y = gen_init_data(20)

In [32]:
bounds = torch.tensor([[0.], [10.]])

In [33]:
#pip install gpytorch

In [34]:
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

In [35]:
single_model = SingleTaskGP(init_x, init_y)
mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)

/usr/local/lib/python3.10/dist-packages/botorch/models/gp_regression.py:161: UserWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning:

Input data is not standardized (mean = tensor([0.9829]), std = tensor([0.0477])). Please consider scaling the input to zero mean and unit variance.



In [36]:
from botorch import fit_gpytorch_model
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [39]:
from botorch.acquisition.monte_carlo import qExpectedImprovement

EI = qExpectedImprovement(model= single_model,
                          best_f= best_init_y)

from botorch.optim import optimize_acqf
candidates, _ = optimize_acqf(acq_function=EI,
                              bounds= bounds,
                              q=1,
                              num_restarts=200,
                              raw_samples = 512,
                              options={"batch_limit":5, "maxiter":200})
candidates

/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.py:362: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.

/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.py:384: RuntimeWarning:

Optimization failed on the second try, after generating a new set of initial conditions.



tensor([[2.7745]])

In [40]:
# now build a loop function
def get_next_point(init_x, init_y, best_init_y, bounds, n_points=1):
  single_model = SingleTaskGP(init_x, init_y)
  mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
  fit_gpytorch_model(mll)

  EI = qExpectedImprovement(model= single_model,
                          best_f= best_init_y)

  candidates, _ = optimize_acqf(acq_function=EI,
                              bounds= bounds,
                              q=n_points,
                              num_restarts=200,
                              raw_samples = 512,
                              options={"batch_limit":5, "maxiter":200})
  return candidates

In [41]:
get_next_point(init_x, init_y, best_init_y, bounds, n_points=2)

/usr/local/lib/python3.10/dist-packages/botorch/models/gp_regression.py:161: UserWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning:

Input data is not standardized (mean = tensor([0.9829]), std = tensor([0.0477])). Please consider scaling the input to zero mean and unit variance.

/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.py:362: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-06 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), NumericalWarning('A not p.d., added jitter

tensor([[ 2.8239],
        [10.0000]])

In [47]:
n_runs = 10
init_x, init_y, best_init_y = gen_init_data(20)

bounds = torch.tensor([[0.],[10.]])

for i in range(n_runs):
  print(f"Num. of optimization run: {i}")
  new_candidates = get_next_point(init_x, init_y, best_init_y, bounds, 1)
  new_results = target_function(new_candidates).unsqueeze(-1)

  print(f"new candidates are: {new_candidates}")
  init_x = torch.cat([init_x, new_candidates])
  init_y = torch.cat([init_y, new_results])

  best_init_y = init_y.max().item()
  print(f"Best point performs this way: {best_init_y}")


Num. of optimization run: 0


/usr/local/lib/python3.10/dist-packages/botorch/models/gp_regression.py:161: UserWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning:

Input data is not standardized (mean = tensor([0.9795]), std = tensor([0.0490])). Please consider scaling the input to zero mean and unit variance.

/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.py:362: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.

/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.

new candidates are: tensor([[2.7644]])
Best point performs this way: 1.0486096143722534
Num. of optimization run: 1


/usr/local/lib/python3.10/dist-packages/botorch/optim/optimize.py:362: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.



new candidates are: tensor([[3.7154]])
Best point performs this way: 1.0486096143722534
Num. of optimization run: 2


/usr/local/lib/python3.10/dist-packages/botorch/models/gp_regression.py:161: UserWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:173: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning:

Input data is not standardized (mean = tensor([0.9695]), std = tensor([0.0743])). Please consider scaling the input to zero mean and unit variance.



KeyboardInterrupt: ignored